In [13]:
import torch
model = torch.load("beit_base_patch16_224_pt22k_ft22k.pth", map_location='cpu')['model']


In [14]:
model_keys = []
nums = 0.
for key in model.keys():
    if "num_batches_tracked" not in key:
        model_keys.append(key)

In [15]:
import torch
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
weights = []
prefix = "model."
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32))
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix + key.replace("qkv", "q")
        key_k = prefix + key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32))
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    elif "q_bias" in key or "v_bias" in key:
        name2weight["name"] = prefix + key.replace("_bias", ".bias")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32))
        weights.append(name2weight)
    elif "head" in key:
        print(key)
        print(model[key].shape)
    else:
        if "relative_position_index" in key:
            dd = dtype.int32
        if "relative_position_index" in key:
            weight = Parameter(Tensor(model[key].numpy(), dtype.int32))
        else:
            weight = Parameter(Tensor(model[key].numpy(), dtype.float32))
        key = prefix + key
        weights.append({"name": key, "data": weight})
save_checkpoint(weights, "beit_base_patch16_224_pt22k_ft22k.ckpt")



head.weight
torch.Size([21841, 768])
head.bias
torch.Size([21841])
